# 第四章，推导与生成

- 在字典、列表、集合等数据结构中，python提供了推导的写法，简洁地迭代这些结构，并根据迭代结果派生出另一套数据
- 当这种理念运用到函数中时，称为生成器。凡是可是使用迭代器的任务都支持生成器函数。生成器可以提升性能并减少内存用量，让代码更容易读懂

### 第27条，用列表推导取代map与filter

### 第29条，用赋值表达式消除重复代码

In [4]:
stock = {
    'nail':125,
    'screws':35,
    'wingnuts':8,
    'washers':24
}

order = ['screws', 'wingnuts', 'clips']

def get_batches(count, size):
    return count//size

# result = {}
# for name in order:
#     count = stock.get(name, 0)
#     batches = get_batches(count, 8)
#     if batches:
#         result[name] = batches
# print(result)

found = {
    name: batches for name in order if (batches:= get_batches(stock.get(name,0), 8))
}
print(found)

# 赋值表达式会导致操作符左侧的变量泄露到包含这条推导语句的作用域里
# 最好不要泄露循环变量，所以赋值表达式最好只用于推导逻辑的条件之中

{'screws': 4, 'wingnuts': 1}


### 第30条，不要让函数直接返回列表，而是逐个生成列表里的值
包含yield表达式的函数是生成器函数


In [9]:
def index_words_iter(text):
    if text:
        yield 0 
    for index, letter in enumerate(text):
        if letter == ' ':
            yield index + 1

address = 'Four score and seven years ago...'
it = index_words_iter(address)
# print(next(it))
# print(next(it))
# print(next(it))
result = list(it)
print(result)

# 生成器函数的好处是一次值返回一个值，它可以将内存消耗量压得比较低

[0, 5, 11, 15, 21, 27]


### 第31条，谨慎地迭代函数所收到的参数

In [16]:
# 注意：迭代器只能产生一次结果
def index_words_iter(text):
    if text:
        yield 0 
    for index, letter in enumerate(text):
        if letter == ' ':
            yield index + 1

address = 'Four score and seven years ago...'
it = index_words_iter(address)
print(list(it))
print(list(it))    # 第二次不会输出结果

# 如果要多次使用迭代器，要对迭代结果做一次备份，但备份的坏处是要占用大量的内存，而这违背了使用生成器函数的初衷

[0, 5, 11, 15, 21, 27]
[]


In [23]:
# 解决方法一：在每次调用生成器时传入一个lambda表达式，这样每次使用时都会生成一个新的迭代器
def index_words_iter(text):
    if text:
        yield 0 
    for index, letter in enumerate(text):
        if letter == ' ':
            yield index + 1

address = 'Four score and seven years ago...'
it = lambda: index_words_iter(address)
print(list(it()))
print(list(it()))       # 第二次调用相当于生成了新的迭代器

[0, 5, 11, 15, 21, 27]
[0, 5, 11, 15, 21, 27]


In [25]:
# 解决方法二：新建一个容器类，实现迭代器协议
class Index_Word:
    def __init__(self, text) -> None:
        self.text = text
    
    def __iter__(self):
        if self.text:
            yield 0 
        for index, letter in enumerate(self.text):
            if letter == ' ':
                yield index + 1

address = 'Four score and seven years ago...'
it = Index_Word(address)
print(list(it))
print(list(it))        # 相比于直接使用生成器函数，容器类中的迭代协议能够在每次调用时生成新的迭代器


[0, 5, 11, 15, 21, 27]
[0, 5, 11, 15, 21, 27]


### 第32条，考虑使用生成器表达式改写数据量较大的列表推导

In [26]:
# 列表推导可以根据输入序列中的每个元素创建一个包含派生元素的新列表。但如果输入的数据量较大，这样做就会导致内存耗尽而崩溃。
# 生成器表达式是推导机制与生成器机制的拓展，生成器表达式是写在一对圆括号内
#

### 第33条，通过yield from把多个生成器连起来使用

In [2]:
def move(period, speed):
    for _ in range(period):
        yield speed

def pause(delay):
    for _ in range(delay):
        yield 0

def animate():
    yield from move(4, 5.0)
    yield from pause(3)
    yield from move(2, 30.0)

def print_animation(delta):
    print(f'Delta: {delta:.1f}')

def run(func):
    for delta in func():
        print_animation(delta)

run(animate)

Delta: 5.0
Delta: 5.0
Delta: 5.0
Delta: 5.0
Delta: 0.0
Delta: 0.0
Delta: 0.0
Delta: 30.0
Delta: 30.0


### 第36条，考虑用itertools拼装迭代器与生成器